In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
import re
import  warnings
from requests import get
warnings.filterwarnings('ignore')

### Scrapping Title Names and ids

### Collecting the latest movie names
    Getting the list of movies and their title name used in IMDB website. So that we can collect the reviews of the movies.

In [2]:
def get_movies(movie_soup):
    name_container = movie_soup.find_all('tbody',class_ = 'lister-list')
    movie_names = name_container[0].children
    movies_list1 = []
    titles_name1 = []
    for name in movie_names:
        try:
            find_name = name.find('td',class_='titleColumn')
            movie_name = find_name.a.text
            n1 = find_name.a.attrs
            movies_list1.append(movie_name)
            titles_name1.append(n1['href'])
        except:
            pass
    return movies_list1,titles_name1

    Here we are collecting the names of the top and low rated movies with their titles mentioned under the imdb website.   

In [3]:
url_list = ['https://www.imdb.com/chart/moviemeter/?ref_=nv_mv_mpm',
           'https://www.imdb.com/chart/bottom?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=cb6cf75a-1a51-49d1-af63-8202cfc3fb01&pf_rd_r=M57YT72578B4NQ2VP7F8&pf_rd_s=right-4&pf_rd_t=15506&pf_rd_i=boxoffice&ref_=chtbo_ql_7']
#div_name = ['lister-list', 'lister']
movies_list = []
titles_name = []
for url in url_list:
    response = get(url)
    movie_soup = BeautifulSoup(response.text, 'html.parser')
    movie_1,title_1 = get_movies(movie_soup)
    movies_list += movie_1
    titles_name += title_1

In [4]:
data = {'movie_name':movies_list,'movie_title':titles_name}
data_m = pd.DataFrame(data)
data_m

,movie_name,movie_title
0,The Wrong Missy,/title/tt9619798/
1,Capone,/title/tt6199572/
2,The King of Staten Island,/title/tt9686708/
3,Extraction,/title/tt8936646/
4,Scoob!,/title/tt3152592/
...,...,...
195,Beverly Hills Chihuahua,/title/tt1014775/
196,Exorcist II: The Heretic,/title/tt0076009/
197,Recep Ivedik 5,/title/tt6439558/
198,The Cat in the Hat,/title/tt0312528/


In [5]:
len(movies_list) ==len(titles_name)

True

    Collecting reviews from the above collected movies

In [6]:
def get_reviews(html_soup):
    review_container = html_soup.find_all('div', class_ = 'lister-list')
    review_list = review_container[0].children
    reviews_text = []
    ratings = []
    #review_list = review_container[0].children
    for review in review_list:
        try:
            rating = review.find('span').text
            text = review.find('div', attrs = {'class': 'text show-more__control'}).text
            reviews_text.append(text)
            ratings.append(rating)
        except:
            pass
    return reviews_text,ratings

In [7]:
size_data = data_m.shape[0]
collect_reviews = []
collect_ratings = []
for i in range(size_data):
    title_code = data_m['movie_title'].values[i]
    url = 'https://www.imdb.com'+title_code+'reviews?ref_=tt_urv'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    collect_review_1,collect_rating_1 = get_reviews(html_soup)
    collect_reviews += collect_review_1
    collect_ratings += collect_rating_1

    The reviews collected so far

In [8]:
len(collect_reviews)

4705

In [26]:
data_movie = pd.DataFrame({'movie_reviews':collect_reviews,'movie_ratings':collect_ratings})
data_movie

,movie_reviews,movie_ratings
0,"Not every movie is supossed to be Amarcord, ca...",\n\n\n\n\n\n7/10\n
1,"So this was never meant to win an Oscar, let's...",\n\n\n\n\n\n8/10\n
2,The names David Spade and Rob Schneider were e...,\n\n\n\n\n\n8/10\n
3,The Wrong Missy (3 out of 5 stars).The Wrong M...,\n\n\n\n\n\n6/10\n
4,David Spade in his trademark role-straight man...,\n\n\n\n\n\n7/10\n
...,...,...
4700,i am pretty sure that no one will read this bu...,\n\n\n\n\n\n2/10\n
4701,I first watched this as a young lad when it ca...,\n\n\n\n\n\n8/10\n
4702,No! Just no! Hell no! No! An abomination! No! ...,\n\n\n\n\n\n1/10\n
4703,I don't see why everyone hates this movie. All...,\n\n\n\n\n\n10/10\n


In [28]:
data_movie['movie_ratings'] = data_movie['movie_ratings'].apply(lambda x:x.replace('\n','')[:-3])
data_movie['movie_ratings']

0        7
1        8
2        8
3        6
4        7
        ..
4700     2
4701     8
4702     1
4703    10
4704     2
Name: movie_ratings, Length: 4705, dtype: object

In [29]:
data_movie['movie_ratings'].unique()

array(['7', '8', '6', '9', '2', '3', '10', '4', '1', '5', 'ripleyspu',
       'DVD_Connoiss', 'Jane-g', 'random-70', 'southbankcin', 'federov',
       'FredGrim', 'James-', 'Chrysanthe', 'george.schm', 'Big Movie ',
       'tfrizz', 'ttss-86', 'Michael_Elli', 'fernandez_1', 'matt',
       'unbrokenme', 'Vartiai', 'harry_tk_y', 'Special-', 'CinemaCl',
       'speedr', 'Dougla', 'sumtim3s', 'CalR', 'alan-chan-158-451',
       'calcman', 'flintknapper_', 'Gordon', 'CraZY_mOVIe_MarSH',
       'andy_a', 'DylanMcChil', 'jimcisne', 'go', 'payroll-05',
       'nicholas_cla', 'scmovie', 'AvinashPata', 'ktul', 'webspinner',
       'SpanishRi', 'knighteagle-111-219', 'hurricanema', 'kevin-miller',
       'neil-', 'Dash1', 'LloydBa', 'krieg8814', 'raz', 'aera',
       'anshuagrawal-23', 'phollyj', 'guybrush', 'Gatorm', 'verbal',
       'leopol', 'imdbmovie', 'pau', 'copperc', 'moondogg', 'cjs',
       'jbdorsk', 'minionl', 'Rina', 'Scarle', 'Alexander Christie-Mil',
       'horrorg', 'Skazii', 'fi

In [30]:
rating_list = ['8', '7', '6','9', '10', '5','2', '4', '3', '1']
data_movie[data_movie['movie_ratings'].isin(rating_list)].shape[0]

4380

    Some of the reviews doesn't have ratings. So we are throwing that reviews into test_data. After training our data, we can predict this reviews rating

In [31]:
train_data = data_movie[data_movie['movie_ratings'].isin(rating_list)] 
test_data = data_movie[~data_movie['movie_ratings'].isin(rating_list)]

In [34]:
train_data['movie_ratings'] = train_data['movie_ratings'].astype(int)

    Here I am labeling the data into 2 parts  and 1. Where 1 indicates the rating of the review > 5 and 0 indicates the rating of the review < 0. Considering this reviews as positive(1) and negative(0)

In [36]:
train_data['label'] = train_data['movie_ratings'].apply(lambda x: 1 if(x>5) else 0)
train_data['label']

0       1
1       1
2       1
3       1
4       1
       ..
4700    0
4701    1
4702    0
4703    1
4704    0
Name: label, Length: 4380, dtype: int64

In [37]:
train_data['label'].value_counts()

0    2286
1    2094
Name: label, dtype: int64

In [76]:
X = train_data['movie_reviews'].values
y = train_data['label'].values

In [77]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X,y, test_size = 0.25, random_state = 3)

In [78]:
X_train[5]

"I had never heard of Chun Li until I first saw the announcement that Kristin Kreuk would get the lead. What I eventually learned about The Street Fighter, I liked. A female lead in an action role which could compare somewhat to Batman sounded enticing. I guess another comparison that crossed my mind was Lara Croft.Kristin Kreuk won a great part when she got the role of Chun Li. Being a fair-weather fan of Smallville, I had long hoped she would find a fitting character in the movies. Her time has come. I like her ability to show intensity and weakness. Emotion is her best partner in acting.As far as the movie itself goes, I felt somewhat cheated. I was intrigued by the introduction which was narrated nicely by Kristin. Some of the narration that continued was OK, but it soon became an awkward and overbearing distraction. The director obviously did not recognize that crossing the line by over-narration in an action movie could be a definite drawback. He also did not seem to know that, w

In [79]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/varshan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Cleaning the reviews

In [80]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def text_prepare(text):
    text = text.lower() 
    text = REPLACE_BY_SPACE_RE.sub(' ', text) 
    text = BAD_SYMBOLS_RE.sub('', text) 
    text = ' '.join([x for x in text.split() if x and x not in STOPWORDS]) 
    return text

In [81]:
X_train = [text_prepare(x) for x in X_train]
X_valid = [text_prepare(x) for x in X_valid]

In [82]:
X_train[5]

'never heard chun li first saw announcement kristin kreuk would get lead eventually learned street fighter liked female lead action role could compare somewhat batman sounded enticing guess another comparison crossed mind lara croftkristin kreuk great part got role chun li fairweather fan smallville long hoped would find fitting character movies time come like ability show intensity weakness emotion best partner actingas far movie goes felt somewhat cheated intrigued introduction narrated nicely kristin narration continued ok soon became awkward overbearing distraction director obviously recognize crossing line overnarration action movie could definite drawback also seem know movie fictitious crime street fighter dont litter kind movie weak poorly played characters like charlie nash detective maya sunee seemed square pegs forced round holes chris klein overplayed part though thought playing immature detective meant acting like 12 year old one moment 17 otherwise sure 12 17 yearolds pla

In [83]:
from collections import Counter
words_counts = Counter()

for words in X_train:
    for word in words.split():
        words_counts[word] += 1

### The top most used words 

In [84]:
sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:3]

[('movie', 8186), ('film', 5060), ('one', 3085)]

### TF-IDF

In [85]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1, 2))

In [86]:
X_train = tfidf_vectorizer.fit_transform(X_train)

In [87]:
X_valid = tfidf_vectorizer.transform(X_valid)

In [88]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(penalty='l2', C = 2, solver = 'liblinear')

In [89]:
lr.fit(X_train,y_train)

LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [90]:
y_pred = lr.predict(X_valid)

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid,y_pred)

0.8611872146118722

In [92]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_valid,y_pred)

array([[500,  72],
       [ 80, 443]])

### Working on the test data

In [99]:
X_test = test_data['movie_reviews'].values
X_test = [text_prepare(text) for text in X_test]

In [100]:
X_test = tfidf_vectorizer.transform(X_test)

In [101]:
test_p = lr.predict(X_test)

In [102]:
test_data['ratings'] = test_p

In [103]:
test_data

,movie_reviews,movie_ratings,ratings
30,"I admired the film more than I enjoyed it, but...",ripleyspu,1
53,"""Extraction"" is fast, vicious and simple. With...",DVD_Connoiss,1
119,Whoa... hold on guys... 53???Who rate this tur...,Jane-g,0
151,Seriously I am not politically correct woke or...,random-70,0
189,"Parasite (Gisaengchung), the 2019 film by Bong...",southbankcin,1
...,...,...,...
4659,This movie isn't entertaining. It's that simpl...,cash76,0
4661,For any forty-something pseudo-Boomer like mys...,Rips,0
4681,Some people claim this is a bad movie... true....,dtm,0
4693,It's really dangerous to make a movie out of a...,Wizar,0


### Checking on the test data

#### Checking random review which is classified as positive. And the text is positive

In [105]:
test_data[test_data['ratings']==1]['movie_reviews'].values[0]

'I admired the film more than I enjoyed it, but it kept my interest throughout despite being a bit of slog at times. It\'s fascinating. The title \'Capone\' invokes the expectation that this was a biopic of the famous gangster, but this isn\'t that at all. This is something very different, and very deliberately so, but due to the title it\'s going to confuse and disappoint most people (which is why Trank hated that he had to change it from the original title \'Fonzo\' for marketing reasons). There\'s hardly a reference for this kind of film; it\'s not a traditional narrative in any sense.What I keep thinking about is how even most gangster films who pretend to de-glamorize criminals and their lifestyle fail; it\'s the same problem with the "anti-" war movie that still depicts a rousing, exciting adventure next to the misery and the atrocities and thus defeats its own purpose. Rare is the war or gangster film that actually achieves to show the underlying emptiness; the ugly, banal, vulg

#### Checking random review which is classified as negative. And the text is negative

In [107]:
test_data[test_data['ratings']==0]['movie_reviews'].values[3]

'Not sure what this is supposed to be. Its definitely not a horror movie. Its not even that scary, has a few unsettling moments but thats it. Its just weird and doesnt really have any dramatic arc or raising the tension. Basically the whole movie is a few strangers saying WTF? constantly and everyone else telling them " \'ts ok ! guys"\nAlso besides that, its revolves around a traumatized, annoying and totally without self respect girl that slowly starts benefiting from this weird place. Because of course, we cannot have a film these days without it being a "personal journey" of some sorts. Writers trying to be smart and "deep".\nYeah nothing beats a bizzare and crazy rituals to make you sane.If youre expecting horror and monsters or supernatural, keep going. Its just a weird film with no real ending, resolution, interesting character or events. Albeit with nice filmography and quite good actors it has no real dramatic structure or captivating story.'